In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import scipy.linalg
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [2]:
import sys
sys.path.append('../')
sys.path.append('../model')
# import model
import UnicycleModel
import cost
from iLQR import iLQR
from scipy.integrate import solve_ivp

In [3]:
x_t = np.zeros(3)
x_t[0] = 0.0
x_t[1] = 2.0
x_t[2] = 0.0
ix = 3
iu = 2
N = 500
delT = 0.1
myModel = UnicycleModel.unicycle('Hello',ix,iu,linearization="numeric_central")
myCost = cost.unicycle('Hello',x_t,N)

In [6]:
maxIter= 100

x0 = np.zeros(3)
x0[0] = -1.0 # -2.0
x0[1] = 0.0 # -0.5
x0[2] = np.pi/2

u0 = np.random.rand(N,iu)
i1 = iLQR('unicycle',delT,N,maxIter,myModel,myCost,discretization="Euler")
x, u, Quu_save, Quu_inv_save, L, l = i1.update(x0,u0)

iteration   cost        reduction   expected    gradient    log10(lambda)
0           378.802     231         240         0.497       0.0         
1           147.896     19          25.7        0.223       -0.2        
2           128.874     14.9        15.8        0.225       -0.6        
3           114.002     12.5        28.2        0.325       -1.2        
4           101.47      18.3        15.8        0.152       -2.0        
5           83.1441     8.56        8.26        0.177       -3.1        
6           74.5799     11.2        11.8        0.214       -4.3        
7           63.4029     8.92        10.1        0.197       -5.7        
8           54.4814     8.59        8.84        0.187       -7.3        
9           45.8953     7.79        7.57        0.177       -inf        


/Users/taewankim/Documents/Code/Personal/iLQR/notebooks/../iLQR.py:319: RuntimeWarning: divide by zero encountered in log10
  self.dlamda = np.minimum(self.dlamda / self.lamdaFactor, 1/self.lamdaFactor)


10          38.1022     7.59        9.37        0.16        -inf        
11          30.5164     4.64        4.57        0.112       -inf        
12          25.8737     3.36        4.27        0.112       -inf        
13          22.5156     4.27        3.49        0.0862      -inf        
14          18.243      0.183       3.49        0.104       -inf        
15          18.0602     5.39        4.51        0.0941      -inf        
16          12.6745     2.02        1.94        0.0662      -inf        
17          10.6553     0.203       1.22        0.0396      -inf        
18          10.4524     1.25        1.27        0.0624      -inf        
19          9.19773     0.000873    0.000924    0.000913    -inf        
SUCCEESS : gradient norm < tolGrad


In [ ]:
plt.figure(figsize=(10,10))
fS = 18
plt.subplot(221)
plt.plot(x[:,0], x[:,1], linewidth=2.0)
plt.plot(x_t[0],x_t[1],"o",label='goal')
plt.gca().set_aspect('equal', adjustable='box')
plt.axis([-2, 2, 0, 4.0])
plt.xlabel('X (m)', fontsize = fS)
plt.ylabel('Y (m)', fontsize = fS)
plt.subplot(222)
plt.plot(np.array(range(N+1))*0.1, x[:,0], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x1 (m)', fontsize = fS)
plt.subplot(223)
plt.plot(np.array(range(N+1))*0.1, x[:,1], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x2 (m)', fontsize = fS)
plt.subplot(224)
plt.plot(np.array(range(N+1))*0.1, x[:,2], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x3 (rad)', fontsize = fS)
plt.legend(fontsize=fS)
plt.show()

plt.figure()
plt.subplot(121)
plt.plot(np.array(range(N))*0.1, u[:,0], linewidth=2.0)
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('v (m/s)', fontsize = fS)
plt.subplot(122)
plt.plot(np.array(range(N))*0.1, u[:,1], linewidth=2.0)
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('w (rad/s)', fontsize = fS)
plt.show()



# Test integration

In [ ]:
x_test = np.zeros_like(x)

In [ ]:
# using for loop
x_test[0,:] = x[0,:]
state_init = x[0,:]
start = time.time()
for i in range(N) :
    state_init = x_test[i,:]
    int_result = solve_ivp(lambda t,x: myModel.forwardDyn(x,u[i,:],idx=t,discrete=False),
                          t_span=(0,0.1),y0=state_init,max_step=1e-2)
    x_test[i+1,:] = int_result.y[:,-1]
print(time.time()-start)

In [ ]:

# parallel integration
def input_by_time(u,t,t_final) :
    if t >= t_final :
        return u[-1,:]
    else :
        return u[int(t*10),:]
t_eval = np.array(range(N+1))*0.1
state_init = x[0,:]
start = time.time()
int_result = solve_ivp(lambda t,x: myModel.forwardDyn(x,input_by_time(u,t,50),idx=t,discrete=False),
                      t_span=(t_eval[0],t_eval[-1]),
                       t_eval = t_eval,
                       y0=state_init,
                       max_step=1e-2,
                      )
x_test_p = int_result.y.transpose()
print(time.time()-start)

In [ ]:
%matplotlib qt
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.plot(np.array(range(N+1))*0.1, x[:,0], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,0], linewidth=2.0,label='ode')
plt.plot(np.array(range(N+1))*0.1, x_test_p[:,0], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x1 (m)', fontsize = fS)
plt.subplot(132)
plt.plot(np.array(range(N+1))*0.1, x[:,1], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,1], linewidth=2.0,label='ode')
plt.plot(np.array(range(N+1))*0.1, x_test_p[:,1], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x2 (m)', fontsize = fS)
plt.subplot(133)
plt.plot(np.array(range(N+1))*0.1, x[:,2], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,2], linewidth=2.0,label='ode')
# plt.plot(np.array(range(N+1))*0.1, x_test_p[:,2], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x3 (rad)', fontsize = fS)
plt.legend(fontsize=fS)
plt.show()